# **Practica 2.2**
#### _Alberto García Doménech - Pablo Daurell Marina_ (Grupo 10)
***

## Parte 2 (Recuperación de información)

In [13]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np

Para esta práctica vamos a utilizar el dataset ```20 Newsgroup``` de Scikit-learn, que contiene textos de un foro sobres distintos temas muy variados.

In [14]:
# Cargamos el dataset
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

print("Training texts:", len(train_data.data))
print("Test texts:", len(test_data.data))

Training texts: 11314
Test texts: 7532


Podemos ver algunos ejemplos de los textos que hay en el dataset:

In [15]:
print('MENSAJE:\n',train_data.data[120])
print('CLASE: ', train_data.target_names[train_data.target[120]])

MENSAJE:
 From: shd2001@andy.bgsu.edu (Sherlette Dixon)
Subject: Christianity & Atheism:  an update
Organization: BGSU
Lines: 32

First, I would like to thank all who sent me their opinions on the matter
at hand.  All advice was taken to heart, if not directly used.  My friend
found out about the matter quite accidently.  After reading some of my
mail, I quit from the mail reader & went about my business.  I must have
trashed my mail improperly, because he got on the same terminal the next
day & saw my old messages.  He thought they were responses to a post he
placed in alt.atheism earlier that week, so he read some of them before
realizing that they were for me.  I got a message from him the next day; he
apologized for reading my mail & said that he did not want to appear to be
a snoop.  He said that he would be willing to talk to me about his views &
didn't mind doing so, especially with a friend.  So we did.  I neither
changed his mind nor did he change mine, as that was not the poi

In [16]:
print('MENSAJE:\n',train_data.data[42])
print('CLASE: ', train_data.target_names[train_data.target[42]])

MENSAJE:
 From: ab245@cleveland.Freenet.Edu (Sam Latonia)
Subject: Re: Need phone number for Western Digital (ESDI problem)
Organization: Case Western Reserve University, Cleveland, Ohio (USA)
Lines: 5
NNTP-Posting-Host: slc10.ins.cwru.edu


Western Digital 1-800-832-4778.....Sam
-- 
Gosh..I think I just installed a virus..It was called MS DOS6...
Don't copy that floppy..BURN IT...I just love Windows...CRASH...

CLASE:  comp.sys.ibm.pc.hardware


### A) Bolsa de palabras binaria

Convertimos el conjunto de datos en una bolsa de palabras, binaria y con n-gramas (1,1), usaremos como vocabulario el diccionario ```words.txt``` (https://github.com/dwyl/english-words/blob/master/words.txt)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
# Pasamos el diccionario a una lista
with open('words.txt') as f:
    dictionary = f.read().splitlines()

# Creamos la bolsa de palabras
vectorizer = CountVectorizer(binary=False, ngram_range=(1,1), vocabulary=dictionary, stop_words='english')  

# Entrenamos al modelo
train_vector_data = vectorizer.fit_transform(train_data.data)

feature_names = vectorizer.get_feature_names()
print(len(feature_names))

466551


In [19]:
np.shape(train_vector_data)

(11314, 466551)

In [20]:
def select_messages(data, target, target_names, n):
    '''
    Selecciona n mensajes de cada clase
    Return: Matriz con una fila por cada clase y tres mensajes por cada fila
    '''
    messages = []
    # Recorremos todas las clases
    for i in range(len(target_names)):
        # Seleccionamos las posiciones correspondientes a los mensajes de la clase i
        indices = np.asarray(np.where(target == i)).ravel()
        # Cogemos los n primeros mensajes de la clase i
        aux = np.take(data, indices)[0:n]
        messages.append(aux)
    
    return np.array(messages)

def vectorize_messages(data, vectorizer):
    '''
    Recibe una matriz de textos y convierte cada texto a una bolsa de palabras,
    usando un vectorizador previamente entrenado
    Return: Matriz con una fila por cada clase y en cada fila otra matriz con los mensajes vectorizados
    '''
    # Recorremos todas las filas (una por cada clase)
    vector_data = []
    for i in range(np.shape(data)[0]):
        aux = vectorizer.transform(data[i, :])
        vector_data.append(aux)
        
    return np.array(vector_data)

In [21]:
# Seleccionamos, del conjunto de test, 3 mensajes de cada una de las 20 clases:
messages = select_messages(test_data.data, test_data.target, test_data.target_names, 3)

In [22]:
# Vectorizamos los mensajes seleccionados
vector_messages = vectorize_messages(messages, vectorizer)

In [23]:
np.shape(vector_messages[0])[0]

3

#### Borrar cuando este hecho vvv
Considera una clase, por ejemplo, politics.

1. Para cada uno de los mensajes de test de la clase politics se calcula la similitud del coseno con todos y cada uno los mensajes de entrenamiento.
2. Se ordenan dichas distancias de mayor y menor y se cogen las X primeras, que son las X que más se parecen. Recuperamos los mensajes correspondientes a las X mayores similitudes del coseno.
3. Se identifica la clase de los X primeros mensajes y se calcula la precisión, contando el porcentaje de ellos que son de la clase politics.
4. Los pasos 1-3 se repiten para los tres mensajes de test de la clase politics. Una vez calculadas las tres precisiones a nivel X, se calcula su media, que es lo que se nos está pidiendo.
5. Los pasos 1-4 debes repetirlos para los distintos niveles de precisión X que se pide.

Todo el proceso debes repetirlo para cada una de las clases, es decir, de los grupos de noticias.
#### Borrar cuando este hecho ^^^

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_precission(data, target, messages, target_names, X):
    '''
    Calcula la similitud del coseno de cada mensaje con todos los mensajes de "data" y
    y calcula la precision de cada clase, a un nivel de exhaustividad X.
    '''
    for i in range(len(messages)): # Recorremos cada clase
        precission = 0
        for m in vector_messages[i]: # Recorremos cada mensaje
            # Calculamos la similitud del coseno de m con los textos de entrenamiento
            cos_sim = cosine_similarity(m, data).ravel()

            # Ordenamos los resultados y cogemos los indices de los X mayores resultados
            max_args = np.flip(np.argsort(cos_sim))[0:X]

            #  max_values = np.take(cos_sim, max_args)         
            #  Recuperamos los mensajes con mayor similitud
            # retrieved_docs = np.take(data, max_args)

            # Identificamos la clase de los mensajes con mayor similitud
            retrieved_classes = np.take(target, max_args)

            # Calculamos la precision (%) para este mensaje y lo acumulamos a las anteriores
            precission += (np.count_nonzero(retrieved_classes == i) / len(retrieved_classes)) * 100


        # Calculamos la media de los porcentajes obtenidos
        precission /= np.shape(vector_messages[i])[0]
        
        print('Class: {} '.format(i), target_names[i])
        print('Precission: ', precission)

In [27]:
# Precision con nivel de exhaustividad = 3
cosine_similarity_precission(train_vector_data, train_data.target, vector_messages, train_data.target_names, 3)

Class: 0  alt.atheism
Precission:  33.333333333333336
Class: 1  comp.graphics
Precission:  22.222222222222218
Class: 2  comp.os.ms-windows.misc
Precission:  22.222222222222218
Class: 3  comp.sys.ibm.pc.hardware
Precission:  55.55555555555555
Class: 4  comp.sys.mac.hardware
Precission:  0.0
Class: 5  comp.windows.x
Precission:  33.33333333333333
Class: 6  misc.forsale
Precission:  77.77777777777777
Class: 7  rec.autos
Precission:  33.333333333333336
Class: 8  rec.motorcycles
Precission:  88.88888888888887
Class: 9  rec.sport.baseball
Precission:  77.77777777777777
Class: 10  rec.sport.hockey
Precission:  66.66666666666667
Class: 11  sci.crypt
Precission:  55.55555555555554
Class: 12  sci.electronics
Precission:  44.444444444444436
Class: 13  sci.med
Precission:  77.77777777777777
Class: 14  sci.space
Precission:  77.77777777777777
Class: 15  soc.religion.christian
Precission:  77.77777777777777
Class: 16  talk.politics.guns
Precission:  55.55555555555554
Class: 17  talk.politics.mideast

In [26]:
# Precision con nivel de exhaustividad = 10
cosine_similarity_precission(train_vector_data, train_data.target, vector_messages, train_data.target_names, 10)

Class: 0  alt.atheism
Precission:  23.333333333333332
Class: 1  comp.graphics
Precission:  20.0
Class: 2  comp.os.ms-windows.misc
Precission:  30.0
Class: 3  comp.sys.ibm.pc.hardware
Precission:  50.0
Class: 4  comp.sys.mac.hardware
Precission:  6.666666666666667
Class: 5  comp.windows.x
Precission:  30.0
Class: 6  misc.forsale
Precission:  56.666666666666664
Class: 7  rec.autos
Precission:  30.0
Class: 8  rec.motorcycles
Precission:  73.33333333333333
Class: 9  rec.sport.baseball
Precission:  76.66666666666667
Class: 10  rec.sport.hockey
Precission:  56.666666666666664
Class: 11  sci.crypt
Precission:  36.666666666666664
Class: 12  sci.electronics
Precission:  50.0
Class: 13  sci.med
Precission:  53.333333333333336
Class: 14  sci.space
Precission:  73.33333333333333
Class: 15  soc.religion.christian
Precission:  50.0
Class: 16  talk.politics.guns
Precission:  40.0
Class: 17  talk.politics.mideast
Precission:  26.666666666666668
Class: 18  talk.politics.misc
Precission:  23.33333333333

Hay una gran diferencia entre los valores de precision medios de las diferentes clases, siendo la mayor diferencia de un 88.88% con un nivel de exhaustividad 3 y de un 70% con un nivel de exhaustividad de 10. Esta clara diferencia puede venir por la ausencia de terminos especificos de la clase que estamos buscando en las consultas lo que puede llevar a una confusión entre clases. Por ejemplo, la clase con menor precisión es _comp.sys.mac.hardware_ con un 0% de precisión con nivel 3 de exhaustividad y un 6.67% con nivel 10, este nivel tan bajo de precisión seguramente se deba a que las consultas elegidas se confunden con otras clases que traten casi el mismo tema, como es el caso de _comp.sys.ibm.pc.hardware_.
Mientras que las clases que mejor precisión media tienen (_rec.motorcycles , rec.sport.basketball_) seguramente se deba a que las consultas tienen términos más específicos que las clasifiquen como las clases a las que pertenecen.

In [40]:
def worst_class_precission(data, target, messages, target_names, X):
    '''
    Calcula la similitud del coseno de cada mensaje con todos los mensajes de "data" y
    y calcula la precision de cada clase, a un nivel de exhaustividad X.
    '''
    for i in range(len(messages)): # Recorremos cada clase SOLO LA QUE PEOR PRECISION TIENE
        precission = 0
        for m in vector_messages[i]: # Recorremos cada mensaje
            # Calculamos la similitud del coseno de m con los textos de entrenamiento
            cos_sim = cosine_similarity(m, data).ravel()

            # Ordenamos los resultados y cogemos los indices de los X mayores resultados
            max_args = np.flip(np.argsort(cos_sim))[0:X]

            #  max_values = np.take(cos_sim, max_args)         
            #  Recuperamos los mensajes con mayor similitud
            retrieved_docs = np.take(data, max_args)

            # Identificamos la clase de los mensajes con mayor similitud
            retrieved_classes = np.take(target, max_args)

            # Calculamos la precision (%) para este mensaje y lo acumulamos a las anteriores
#             precission += (np.count_nonzero(retrieved_classes == i) / len(retrieved_classes)) * 100


#     mostramos los mensajes con mayor similitud
        print(retrieved_docs)


        # Calculamos la media de los porcentajes obtenidos
#         precission /= np.shape(vector_messages[i])[0]
#         print('Class: {} '.format(i), target_names[i])
#         print('Precission: ', precission)

SyntaxError: invalid syntax (<ipython-input-40-55ef48d47d0b>, line 6)